In [1]:
import pandas as pd
import numpy as np

# 데이터 정렬
df = pd.read_csv("./data/405957090_5천만건(hum,voc포함).csv")
df["updated"] = pd.to_datetime(df["updated"])
df = df.sort_values("updated")

# -1이 있는 행을 삭제
df = df.replace(-1, np.nan)  # -1을 NaN으로 변경
df = df.dropna()  # NaN이 있는 행 삭제

# 온도가 1500 이상 4500 이하인 데이터만 선택
df = df[(df['temperature'] >= 1500) & (df['temperature'] <= 4500)]

# 날짜 정상 데이터 선택
df = df[(df['days'] > 0) & (df['days'] <= 1800)]

# 성별 정상 데이터 선택
df = df[df["sex"]!=-1]

# 4만 이내, 선형
d1 = [2179,1766,2220,1970,1702,2411,357,2552,1946,2273,1416]
df = df[df["device_id"].isin(d1)]


# 고유한 디바이스 아이디들을 가져오기
unique_device_ids = df['device_id'].unique()

# 결과를 저장할 빈 데이터프레임 생성
sec_df = pd.DataFrame()

# 각 디바이스 아이디에 대해 데이터프레임 생성 및 병합
for target_device_id in unique_device_ids:

    one_df = df[df['device_id'] == target_device_id]

    # 선택한 device_id의 시작일과 마지막일 구하기
    start_date = one_df['updated'].min()
    end_date = one_df['updated'].max()

    # 해당 기간 동안의 모든 초를 가진 데이터프레임 생성
    all_seconds_df = pd.DataFrame({'updated': pd.date_range(start=start_date, end=end_date, freq='S')})

    # 기존 데이터프레임과 합치기
    result_df = pd.merge(all_seconds_df, one_df, on='updated', how='left')

    # _id 열은 불필요하므로 제거
    result_df = result_df.drop('_id', axis=1)

    # null로 초기화 (temperature, humidity, voc만 null로, 나머지는 ffill로 채우기)
    result_df[['updated','device_id','days','sex']] = result_df[['updated','device_id','days','sex']].ffill()

    # 최종 결과에 현재 결과 추가
    sec_df = pd.concat([sec_df, result_df])

# 데이터 자료형과 인덱스 정리
sec_df.reset_index(drop=True, inplace=True)
sec_df['device_id'] = sec_df['device_id'].astype('int')
sec_df['days'] = sec_df['days'].astype('int')
sec_df['sex'] = sec_df['sex'].astype('int')

In [2]:
# 가장 어린/많은 아이의 days값 추출
days_min_value = sec_df['days'].unique().min()
days_max_value = sec_df['days'].unique().max()

# 가장 어린/많은 아이의 디바이스아이디 추출
d_lo_id = sec_df[sec_df['days']==days_min_value]['device_id'].head(1).iloc[0]
d_up_id = sec_df[sec_df['days']==days_max_value]['device_id'].head(1).iloc[0]

# 가장 어린/많은 아이 df 생성
df_lo = sec_df[sec_df['device_id']==d_lo_id]
df_up = sec_df[sec_df['device_id']==d_up_id]

# # 두 데이터에서 겹치는 날짜만을 추출
# date_list = np.intersect1d(df_lo['updated'].dt.date, df_up['updated'].dt.date)

# df_lo = df_lo[df_lo['updated'].dt.date.isin(date_list)]
# df_up = df_up[df_up['updated'].dt.date.isin(date_list)]

# 인덱스 재정의
df_lo.reset_index(drop=True, inplace=True)
df_up.reset_index(drop=True, inplace=True)

In [ ]:
import matplotlib.pyplot as plt

# 시작일자, 마감일자 지정
start_day = min(df_lo['updated'].min(), df_up['updated'].min())
end_day = max(df_lo['updated'].max(), df_up['updated'].max())

# 날짜 지정
date_list = pd.date_range(start=start_day, end=end_day, freq='D')

for day in date_list:

    # 날짜별 데이터프레임 재정의
    one_df_lo = df_lo[df_lo['updated'].dt.date == day.date()]
    one_df_up = df_up[df_up['updated'].dt.date == day.date()]

    # 두 그래프를 동시에 시각화
    fig, axes = plt.subplots(1,2, figsize=(20,5))
    axes[0].plot(one_df_lo['updated'], one_df_lo['temperature'])
    axes[1].plot(one_df_up['updated'], one_df_up['temperature'])

    axes[0].set_title(f'device_id 2220(Days 3~15) - {day.date()}')
    axes[1].set_title(f'device_id 1416(Days 1520~1604 - {day.date()})')

    # # 축을 24시간으로 고정함
    # for ax in axes:
    #     hours = pd.date_range('00:00', '23:00', freq='H')
    #     ax.set_xlim((hours[0], hours[-1]))
    #     # ax.set_xticklabels([dt.strftime('%H:%M') for dt in hours], rotation=45, ha='right')

    plt.show()


In [16]:
hours = pd.date_range('00:00', '23:00', freq='H')[0].time()
hours

datetime.time(0, 0)